<a href="https://colab.research.google.com/github/Fabriloko/Machine_Learn/blob/main/Lista2_Fabricio_414900_AM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

In [ ]:
mama_dataset = np.genfromtxt('/content/breastcancer.csv',delimiter=',')

# **Funções Uteis**

In [ ]:
def MSE(y,y_pred): # Calcula o MSE
  return np.mean((y - y_pred)**2)

def RMSE(y,y_pred): # Calcula o MSE
  return np.mean(np.sqrt((y - y_pred)**2))

def dividir_treino_teste(dados):
  np.random.shuffle(dados)
  X_treino = dados[:,:-1][:int(len(dados)*0.7)]
  X_teste  = dados[:,:-1][int(len(dados)*0.7):]
  y_treino = dados[:,[-1]][:int(len(dados)*0.7)]
  y_teste  = dados[:,[-1]][int(len(dados)*0.7):]
  return y_treino,y_teste,X_treino, X_teste

def make_meshgrid(x, y, steps=300):
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, steps), np.linspace(y_min, y_max, steps))
    return xx, yy

def plot_contours(ax, clf, xx, yy, colors=['red', 'blue']):
    labels = clf(np.c_[ xx.ravel(), yy.ravel() ]).reshape(xx.shape)
    out = ax.contourf(xx, yy, labels, levels=len(np.unique(labels))-1, colors=colors, alpha=0.5)    
    return out

class normalize_01(): #Normalizacao 0 ou 1
  def __init__(self, X):
    self.min = np.min(X,axis=0)
    self.max = np.max(X,axis=0)
  def norma(self, X):
    return (X - self.min)/(self.max - self.min)
  def desnorm(self, X):
    return X * (self.max - self.min) + self.min

def sig(z): #Funcao Sigmoide
  return 1/(1 + np.exp(-z))

def Gaussian(x, media, cov):
    dim = np.shape(cov)[0]
    # Medidas do Determinante da Matriz de Covariancia
    covdet = np.linalg.det(cov + np.eye(dim) * 0.000001)
    covinv = np.linalg.inv(cov + np.eye(dim) * 0.000001)
    xdiff = (x - media).reshape((1, dim))

    # Funcao Densidade de Probabilidade
    prob = 0.5 * (np.log( 2 * np.pi * np.abs(covdet)) + xdiff @ covinv @ xdiff.T)
    return prob


# **Questão 1:**

## **Avaliacao dos Classificadores**

In [ ]:
#Avaliacao dos Classificadores
def metricas(real,pred):
  count = 0
  verd_posi  = 0 #Verdadeiro Positivos
  falso_neg  = 0 #Falsos Negativos
  falso_posi = 0 #Falsos Positivos
  for i in range(len(pred)):
    if pred[i] == real[i]:
      count += 1 
    if pred[i] == 1 and pred[i] == real[i]:
      verd_posi += 1 
    if pred[i] == 0 and pred[i] != real[i]:
      falso_neg += 1 
    if pred[i] == 1 and pred[i] != real[i]:
      falso_posi += 1 

  acc  = count    / len(pred)
  rev  = verd_posi / (verd_posi + falso_neg) 
  prec = verd_posi / (verd_posi + falso_posi)
  F_1 = 2 * (rev * prec)/(rev + prec)

  aux = np.array([acc, rev, prec, F_1])
  return [acc, rev, prec, F_1]

## **Regressão Logistica**

In [ ]:
def RegLogi(dados): # Regressao Logistica adaptada a partir do SGD  
  w_SGD = np.zeros((1,dados.shape[1])) # Vetor de parametros iniciais
  alpha = 0.1 # Valor de Alpha
  iteracoes = 100 # Numero maximo de iteracoes
  metr = [] #Array de metricas

  kf = KFold(n_splits=10,shuffle=True)
  for treino, teste in kf.split(dados):  
    treino = dados[treino] 
    teste = dados[teste]
    
    X_treino = treino[:,:-1]
    X_teste = teste[:,:-1]
    y_treino = treino[:,[-1]] 
    y_teste = teste[:,[-1]]

    normalize_X = normalize_01(X_treino)
    X_treino = normalize_X.norma(X_treino) 
    X_teste  = normalize_X.norma(X_teste) 
    X_treino = np.c_[np.ones(X_treino.shape[0]),X_treino]
    X_teste  = np.c_[np.ones(X_teste.shape[0]),X_teste]

    for i in range(iteracoes): #Laco de atualizacao
      for j in np.random.permutation(X_treino.shape[0]): #Permutacao aleatoria das entradas
        y_pred = sig(w_SGD @ X_treino[[j]].T) #Saida estimada para y_i
        error = y_treino[j] - y_pred #Atualizacao do Erro para y_i
        w_SGD = w_SGD + alpha * error * X_treino[[j]] # Atualizacao do vetor de parametros   
    
    y_pred = np.round(sig(X_teste @ w_SGD.T)) #Considerando uma probabilidade de 50% para positivo 
    metr.append(metricas(y_teste,y_pred))
  return metr

metr_RL = RegLogi(mama_dataset)

acc  = []
rev  = [] 
prec = []
F_1  = []

for i in range(10):
  acc.append(metr_RL[i][0])
  rev.append(metr_RL[i][1])
  prec.append(metr_RL[i][2])
  F_1.append(metr_RL[i][3])

print("Acuracia: Média = ",np.mean(acc),", Desvio Padrão = ",np.std(acc))
print("Revocação: Média = ",np.mean(rev),", Desvio Padrão = ",np.std(rev))
print("Precisão: Média = ",np.mean(prec),", Desvio Padrão = ",np.std(prec))
print("F-1 score: Média = ",np.mean(F_1),", Desvio Padrão = ",np.std(F_1))


Acuracia: Média =  0.9842105263157894 , Desvio Padrão =  0.012280701754385968
Revocação: Média =  0.9913078149920256 , Desvio Padrão =  0.013317384370015936
Precisão: Média =  0.9833836098541981 , Desvio Padrão =  0.02221330545064317
F-1 score: Média =  0.9870842192633237 , Desvio Padrão =  0.010035232778626293


## **Análise do discriminante Gaussiano**

In [ ]:
def DiscGauss(dados):
  metr = [] #Array de metricas

  kf = KFold(n_splits=10,shuffle=True)
  for treino, teste in kf.split(dados):  
    treino = dados[treino] 
    teste = dados[teste]
    
    X_treino = treino[:,:-1]
    X_teste = teste[:,:-1]
    y_treino = treino[:,[-1]] 
    y_teste = teste[:,[-1]]
    
    # Vetores de Quantidades. Classes(C1 e C0) e Resultados(Positivos / Negativos)
    C1 = 0 
    C0 = 0
    postivos = []
    negetivos = []
    
    # Contando e Agrupando os resultados positivos e negativos
    for (x,y) in zip(X_treino, y_treino):
        if y == 1: # Verdadeiros Positivos
            C1 += 1
            postivos.append(list(x))
        else:      # Verdadeiros Negativos
            C0 += 1
            negetivos.append(list(x))

    # Estimando a probabilidade dos exemplos positivos e negativos
    lin, col = np.shape(X_treino)
    priori_C1 = C1/lin            #Probabiliade da Classe 1 
    priori_C0 = 1 - priori_C1       #Probabiliade da Classe 0

    # Estimando os vetores de media dos exemplos positivos e negativos
    postivos = np.array(postivos)
    negetivos = np.array(negetivos)
    media_positivos = np.sum(postivos, axis= 0) / C1
    media_negetivos = np.sum(negetivos, axis= 0) / C0

    # Estimando a matriz de covariancia
    delta_positivos = postivos - media_positivos
    delta_negativos = negetivos - media_negetivos
    sigma = []
    for delta in delta_positivos:
        delta = delta.reshape(1, col)
        aux = delta.T @ delta
        sigma.append(aux)
    for delta in delta_negativos:
        delta = delta.reshape(1, col)
        aux = delta.T @ delta
        sigma.append(aux)
    sigma = np.array(sigma)
    sigma = np.sum(sigma, axis = 0) / (lin - 1) 
    media_positivos = media_positivos.reshape(1, col)
    media_negetivos = media_negetivos.reshape(1, col)

    #Prevendo novos resultados
    y_pred = []
    for d in X_teste:
        prob_positivo = np.log(priori_C1) - Gaussian(d, media_positivos, sigma)
        prob_negativo = np.log(priori_C0) - Gaussian(d, media_negetivos, sigma)
        if prob_positivo >= prob_negativo:
            y_pred.append(1)
        else:
            y_pred.append(0)
    metr.append(metricas(y_teste, y_pred))
  return metr

metr_GDA = DiscGauss(mama_dataset)

acc  = []
rev  = [] 
prec = []
F_1  = []

for i in range(10):
  acc.append(metr_GDA[i][0])
  rev.append(metr_GDA[i][1])
  prec.append(metr_GDA[i][2])
  F_1.append(metr_GDA[i][3])

print("Acuracia: Média = ",np.mean(acc),", Desvio Padrão = ",np.std(acc))
print("Revocação: Média = ",np.mean(rev),", Desvio Padrão = ",np.std(rev))
print("Precisão: Média = ",np.mean(prec),", Desvio Padrão = ",np.std(prec))
print("F-1 score: Média = ",np.mean(F_1),", Desvio Padrão = ",np.std(F_1))


Acuracia: Média =  0.9560776942355889 , Desvio Padrão =  0.01794749161648426
Revocação: Média =  0.9937254901960785 , Desvio Padrão =  0.012579619554780737
Precisão: Média =  0.9396905292570927 , Desvio Padrão =  0.02917379553481384
F-1 score: Média =  0.9655874775139079 , Desvio Padrão =  0.013564137300766912


## **Naive Bayes Gaussiano**

In [ ]:
def NaiveBayesGauss(dados):
  metr = [] #Array de metricas

  kf = KFold(n_splits=10,shuffle=True)
  for treino, teste in kf.split(dados):  
    treino = dados[treino] 
    teste = dados[teste]
    
    X_treino = treino[:,:-1]
    X_teste = teste[:,:-1]
    y_treino = treino[:,[-1]] 
    y_teste = teste[:,[-1]]
    
    # Vetores de Quantidades. Classes(C1 e C0) e Resultados(Positivos / Negativos)
    C1 = 0 
    C0 = 0
    postivos = []
    negetivos = []
    
    # Contando e Agrupando os resultados positivos e negativos
    for (x,y) in zip(X_treino, y_treino):
        if y == 1: # Verdadeiros Positivos
            C1 += 1
            postivos.append(list(x))
        else:      # Verdadeiros Negativos
            C0 += 1
            negetivos.append(list(x))

    # Estimando a probabilidade dos exemplos positivos e negativos
    lin, col = np.shape(X_treino)
    priori_C1 = C1/lin            #Probabiliade da Classe 1 
    priori_C0 = 1 - priori_C1       #Probabiliade da Classe 0

    # Estimando os vetores de media dos exemplos positivos e negativos
    postivos = np.array(postivos)
    negetivos = np.array(negetivos)
    media_positivos = np.sum(postivos, axis= 0) / C1
    media_negetivos = np.sum(negetivos, axis= 0) / C0

    # Estimando a matriz de covariancia
    delta_positivos = postivos - media_positivos
    delta_negativos = negetivos - media_negetivos
    sigma = []
    for delta in delta_positivos:
        delta = delta.reshape(1, col)
        aux = delta.T @ delta
        sigma.append(aux)
    for delta in delta_negativos:
        delta = delta.reshape(1, col)
        aux = delta.T @ delta
        sigma.append(aux)
    sigma = np.array(sigma)
    sigma = np.sum(sigma, axis = 0) / (lin - 1) 
    media_positivos = media_positivos.reshape(1, col)
    media_negetivos = media_negetivos.reshape(1, col)
    sigma = sigma * np.eye(np.shape(sigma)[0]) # Matriz de Covariancia Diagonal
    
    #Prevendo novos resultados
    y_pred = []
    for d in X_teste:
        prob_positivo = np.log(priori_C1) - Gaussian(d, media_positivos, sigma)
        prob_negativo = np.log(priori_C0) - Gaussian(d, media_negetivos, sigma)
        if prob_positivo >= prob_negativo:
            y_pred.append(1)
        else:
            y_pred.append(0)
    metr.append(metricas(y_teste, y_pred))
  return metr

metr_NBG = NaiveBayesGauss(mama_dataset)

acc  = []
rev  = [] 
prec = []
F_1  = []

for i in range(10):
  acc.append(metr_NBG[i][0])
  rev.append(metr_NBG[i][1])
  prec.append(metr_NBG[i][2])
  F_1.append(metr_NBG[i][3])

print("Acuracia: Média = ",np.mean(acc),", Desvio Padrão = ",np.std(acc))
print("Revocação: Média = ",np.mean(rev),", Desvio Padrão = ",np.std(rev))
print("Precisão: Média = ",np.mean(prec),", Desvio Padrão = ",np.std(prec))
print("F-1 score: Média = ",np.mean(F_1),", Desvio Padrão = ",np.std(F_1))


Acuracia: Média =  0.9384398496240601 , Desvio Padrão =  0.03712300263925304
Revocação: Média =  0.9775721256971257 , Desvio Padrão =  0.026996259004650584
Precisão: Média =  0.9295702329912856 , Desvio Padrão =  0.049644007092126435
F-1 score: Média =  0.9520565760197416 , Desvio Padrão =  0.028356606952434135


## **KNN**

In [ ]:
def KNN(dados, k = 3):
  metr = []

  kf = KFold(n_splits=10,shuffle=True)
  for treino, teste in kf.split(dados):  
    treino = dados[treino] 
    teste = dados[teste]
    
    X_treino = treino[:,:-1]
    X_teste = teste[:,:-1]
    y_treino = treino[:,[-1]] 
    y_teste = teste[:,[-1]]

    y_pred = []
    for x in X_teste:
      distancia = np.sqrt(np.sum(X_treino - x, axis= 1) ** 2)
      aux = np.round(np.mean(y_treino[np.argpartition(distancia, k)][:k]))
      y_pred.append(aux)

    metr.append(metricas(y_teste, y_pred))
  return metr

metr_KNN = KNN(mama_dataset)

acc  = []
rev  = [] 
prec = []
F_1  = []

for i in range(10):
  acc.append(metr_KNN[i][0])
  rev.append(metr_KNN[i][1])
  prec.append(metr_KNN[i][2])
  F_1.append(metr_KNN[i][3])

print("Acuracia: Média = ",np.mean(acc),", Desvio Padrão = ",np.std(acc))
print("Revocação: Média = ",np.mean(rev),", Desvio Padrão = ",np.std(rev))
print("Precisão: Média = ",np.mean(prec),", Desvio Padrão = ",np.std(prec))
print("F-1 score: Média = ",np.mean(F_1),", Desvio Padrão = ",np.std(F_1))

Acuracia: Média =  0.8804824561403508 , Desvio Padrão =  0.04761600743823725
Revocação: Média =  0.9063752835509916 , Desvio Padrão =  0.05723989090763634
Precisão: Média =  0.9059442196864216 , Desvio Padrão =  0.060145803841531266
F-1 score: Média =  0.9036347002941122 , Desvio Padrão =  0.03558505552856812


## **Árvore de decisão**